# Full Spec End-to-End Notebook

本ノートは **src の単一真実ロジック** を呼び出し、以下を1本で実行する。
- データ取得/整形
- STRATEGY_REGISTRY 全strategy計算
- 全比較（NAV/summary）
- attribution（factor/internal/overlay）
- dashboard（代表4戦略）
- 最終結論（Full Spec推奨）

固定デフォルト: `split_date=2021-01-01`, `M=12`, `η=0.2`。


In [ ]:
from __future__ import annotations

import os
import sys
from pathlib import Path

import pandas as pd

try:
    from IPython.display import Image, display
except Exception:  # noqa: BLE001
    Image = None
    def display(x):
        print(x)

def _find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "src" / "thematic_strategy").exists():
            return candidate
    raise FileNotFoundError("Project root not found: src/thematic_strategy が見つかりません")

ROOT = _find_project_root(Path.cwd()).resolve()
(ROOT / "outputs" / "logs" / "matplotlib").mkdir(parents=True, exist_ok=True)
os.environ.setdefault("MPLCONFIGDIR", str(ROOT / "outputs" / "logs" / "matplotlib"))

SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from thematic_strategy import (
    DEFAULT_BASE_PARAMS,
    DEFAULT_REPRESENTATIVE_IDS,
    FULL_SPEC_RECOMMENDED_ID,
    run_full_spec_pipeline,
)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 240)


In [ ]:
BASE_PARAMS = dict(DEFAULT_BASE_PARAMS)
BASE_PARAMS["split_date"] = "2021-01-01"

result = run_full_spec_pipeline(
    base_params=BASE_PARAMS,
    include_optional_t3_on_t2=True,
    reuse_nav_if_exists=True,
    representative_ids=list(DEFAULT_REPRESENTATIVE_IDS),
    recommended_strategy_id=FULL_SPEC_RECOMMENDED_ID,
)

print("best_strategy_id:", result["best_strategy_id"])
print("recommended_strategy_id:", result["recommended_strategy_id"])
print("recommended_is_best:", result["conclusion"]["recommended_is_best"])


In [ ]:
summary_all = result["summary_all"].copy()
summary_period = result["summary_period"].copy()
best_sid = result["best_strategy_id"]

print("=== nav_summary_with_onoff_stepB (上位10) ===")
display(summary_all.head(10))

print("=== best strategy Train/Test/All metrics ===")
display(summary_period[summary_period["strategy_id"] == best_sid])

compare_ids = ["EQW", "T1_base_best", "T1_best_onoff_statepred_M12_eta0p2"]
compare_table = summary_all[summary_all["strategy_id"].isin(compare_ids)].copy()
compare_table = compare_table[["strategy_id", "TrainSharpe", "TestSharpe", "Sharpe", "MDD", "AvgTurnover", "TotalCost"]]
print("=== Full Spec 比較（EQW / T1_base_best / 推奨）===")
display(compare_table.sort_values("TestSharpe", ascending=False))


In [ ]:
print("=== attribution snapshots ===")
display(result["attribution_betas"].head())
display(result["attribution_contrib"].head())
display(result["attribution_internal"].head())
display(result["attribution_overlay"].head())


In [ ]:
print("=== generated figures ===")
paths = result["output_paths"]
figure_paths = [
    paths["nav_linear_fig"],
    paths["nav_log_fig"],
    paths["drawdown_fig"],
    paths["attribution_beta_fig"],
    paths["attribution_contrib_fig"],
    paths["attribution_internal_fig"],
    paths["attribution_overlay_fig"],
] + list(paths["dashboard_paths"])

for p in figure_paths:
    print(p)
    if Image is not None and Path(p).exists():
        display(Image(filename=str(p)))


In [ ]:
c = result["conclusion"]
best_sid = c["best_strategy_id"]

def _fmt(x):
    try:
        return f"{float(x):.4f}"
    except Exception:
        return str(x)

print("=== 最終結論 ===")
print(f"- Full Spec 推奨: T1_best_onoff_statepred_M12_eta0p2")
print(f"- 実測の最良 strategy_id(TestSharpe最大): {best_sid}")
print("- 最良strategy metrics (Train/Test/All):")
print("  Train:", c["best_train_metrics"])
print("  Test :", c["best_test_metrics"])
print("  All  :", c["best_all_metrics"])

beta_delta = c["beta_change_vs_t1base"]
alpha_delta = c["alpha_change_vs_t1base"]
beta_msg = "β低下寄与" if pd.notna(beta_delta) and beta_delta < 0 else "β低下は限定的"
alpha_msg = "α増加寄与" if pd.notna(alpha_delta) and alpha_delta > 0 else "α増加は限定的"
print(f"- 勝因1（β/α）: {beta_msg}, {alpha_msg} (Δβ={_fmt(beta_delta)}, Δα={_fmt(alpha_delta)})")
print(
    f"- 勝因2（寄与分解）: selection={_fmt(c['selection_sum_test'])}, "
    f"weighting={_fmt(c['weighting_sum_test'])}, overlay={_fmt(c['overlay_sum_test'])}"
)
turnover_delta = c["avg_turnover_best"] - c["avg_turnover_t1base"] if pd.notna(c["avg_turnover_t1base"]) else float('nan')
cost_delta = c["total_cost_best"] - c["total_cost_t1base"] if pd.notna(c["total_cost_t1base"]) else float('nan')
print(f"- 副作用（turnover/コスト）: Δturnover={_fmt(turnover_delta)}, Δtotal_cost={_fmt(cost_delta)}")
